In [10]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [11]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    preprocessing_function=preprocess_input, 
    validation_split=0.2
)

In [12]:
train_generator = datagen.flow_from_directory(
    '../data/base',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

Found 1677 images belonging to 13 classes.


In [13]:
val_generator = datagen.flow_from_directory(
    '../data/base',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 416 images belonging to 13 classes.


In [14]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [15]:
base_model.trainable = False

In [16]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(13, activation='softmax')(x)

In [17]:
model = Model(inputs=base_model.input, outputs=output)

In [18]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

/home/sandra/miniconda3/envs/tensorflow/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


I0000 00:00:1750798161.896929  138032 service.cc:148] XLA service 0x77ca5c022430 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1750798161.896944  138032 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-06-24 22:49:21.940448: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1750798162.221386  138032 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-06-24 22:49:22.486074: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1392', 16 bytes spill stores, 16 bytes spill loads

2025-06-24 22:49:22.574875: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1392_0', 112 by

 2/53 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0938 - loss: 2.9961 

I0000 00:00:1750798164.379170  138032 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step - accuracy: 0.1188 - loss: 2.6856

/home/sandra/miniconda3/envs/tensorflow/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


53/53 ━━━━━━━━━━━━━━━━━━━━ 18s 265ms/step - accuracy: 0.1192 - loss: 2.6834 - val_accuracy: 0.1587 - val_loss: 2.4581
Epoch 2/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 11s 202ms/step - accuracy: 0.2325 - loss: 2.2788 - val_accuracy: 0.1947 - val_loss: 2.4431
Epoch 3/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 10s 191ms/step - accuracy: 0.2840 - loss: 2.1949 - val_accuracy: 0.2091 - val_loss: 2.4300
Epoch 4/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 11s 199ms/step - accuracy: 0.3162 - loss: 2.0679 - val_accuracy: 0.2260 - val_loss: 2.4138
Epoch 5/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 10s 197ms/step - accuracy: 0.3378 - loss: 1.9844 - val_accuracy: 0.2596 - val_loss: 2.3995
Epoch 6/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 11s 199ms/step - accuracy: 0.3477 - loss: 1.9279 - val_accuracy: 0.2332 - val_loss: 2.4438
Epoch 7/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 11s 200ms/step - accuracy: 0.3872 - loss: 1.8520 - val_accuracy: 0.2812 - val_loss: 2.3694
Epoch 8/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 11s 204ms/step - accuracy: 0.3929 - loss: 1.8614 - val_accuracy: 0.223